### Ordtyper i handel med finansiella instrument

#### Marknadsorder (Market Order)
En order att köpa eller sälja en finansiell tillgång omedelbart till det aktuella marknadspriset.

#### Begränsad order (Limit Order)
En order att köpa eller sälja en finansiell tillgång till ett specifikt pris eller bättre. 

#### Stop-Order (Stop Order)
En order att köpa eller sälja en finansiell tillgång när den når ett visst pris, kallat stopppris.

#### Stop-Limit-Order
En kombination av en stop-order och en limit-order. När stopppriset nås, konverteras ordern till en limit-order.

#### Fill-Or-Kill (FOK)
En order som måste utföras omedelbart och i sin helhet eller inte alls.

#### Immediate-Or-Cancel (IOC)
En order som måste utföras omedelbart. Alla delar som inte kan fyllas omedelbart kommer att avbrytas.

#### Good-Till-Canceled (GTC)
En order som kommer att vara aktiv tills den utförs eller avbryts av handlaren.

#### Day Order
En order som är aktiv endast under en handelsdag.

#### All-Or-None (AON)
En order där alla delar måste fyllas, eller så fylls inget alls.

#### Iceberg Order
En större order som delas upp i mindre, publika order för att dölja det totala ordervolymen.








Kostnad Nordnet API = 1000 SEK / Månad
Dokumentation som allt bygger på - https://www.nordnet.se/externalapi/docs/api

### Generell info
Kostnad Nordnet API = 1000 SEK / Månad 

Dokumentation som allt bygger på - https://www.nordnet.se/externalapi/docs/api

In [2]:
import requests
import json
import base64
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import padding

In [7]:
class Order:
    def __init__(self, session_id, accid):
        self.session_id = session_id
        self.accid = accid
        self.base_url = f'https://public.nordnet.se/api/2/accounts/{self.accid}/orders'
        encoded_credentials = base64.b64encode(f"{self.session_id}:{self.session_id}".encode("utf-8")).decode("utf-8")
        self.headers = {
            'Authorization': f'Basic {encoded_credentials}'
        }

    def place_order(self, market_id, side, volume, price, order_type='NORMAL', **kwargs):
        url = self.base_url
        data = {
            'market_id': market_id,
            'side': side,
            'volume': volume,
            'order_type': order_type,
            'price' : price
        }
              
        for key, value in kwargs.items():
            data[key] = value
            
        response = requests.post(url, headers=self.headers, data=data)
        return json.loads(response.text)
    
    def get_orders(self, include_deleted=False):
        url = self.base_url
        params = {}
        if include_deleted:
            params['deleted'] = 'true'

        response = requests.get(url, headers=self.headers, params=params)
        return json.loads(response.text)
    
    def delete_order(self, order_id):
        url = f'{self.base_url.format(self.accid)}/{order_id}'
        response = requests.delete(url, headers=self.headers)
        return json.loads(response.text)



In [9]:
class NordnetSession:
    def __init__(self, public_key_path = "nordnet.pem"):
        self.session_key = None
        self.public_key_path = public_key_path

    def encrypt_and_encode_credentials(self, username, password, timestamp):
        credentials = f"{username}:{password}:{timestamp}"
        b64_credentials = base64.b64encode(credentials.encode("utf-8"))
        
        with open(self.public_key_path, "rb") as key_file:
            public_key = serialization.load_pem_public_key(
                key_file.read(),
                backend=default_backend()
            )
        
        encrypted = public_key.encrypt(
            b64_credentials,
            padding.OAEP(
                mgf=padding.MGF1(algorithm="SHA256"),
                algorithm="SHA256",
                label=None
            )
        )
        return base64.b64encode(encrypted)

    def create_session(self, username, password, service_name, timestamp, session_lang="en"):
        encrypted_auth = self.encrypt_and_encode_credentials(username, password, timestamp)
        headers = {"Accept-Language": session_lang}
        form_data = {
            "auth": encrypted_auth,
            "service": service_name,
            "session_lang": session_lang
        }
        
        response = requests.post("https://public.nordnet.se/api/2/login", headers=headers, data=form_data)
        if response.status_code == 200:
            login_response = json.loads(response.text)
            self.session_key = login_response["session_key"]
            return login_response
        elif response.status_code in [400, 401, 429, 503]:
            error_response = json.loads(response.text)
            return error_response
        else:
            return None
